In [ ]:
# Setup: imports and display options
import pandas as pd
import numpy as np
from io import StringIO

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('Pandas version:', pd.__version__)

## Synthetic CSV (Security Events)
To make this notebook self-contained, we'll generate a small CSV that mimics auth or prompt logs.
Columns: `timestamp`, `user`, `source_ip`, `action`, `status`, `latency_ms`, `prompt_tokens`.

In production, you'd point to a real file or object store path.

In [ ]:
# Create a synthetic CSV in-memory and save to disk
csv_data = StringIO('''timestamp,user,source_ip,action,status,latency_ms,prompt_tokens
2025-01-01T10:00:00Z,alice,10.0.0.5,login,success,120,850
2025-01-01T10:00:05Z,bob,10.0.0.6,login,failure,95,620
2025-01-01T10:00:07Z,alice,10.0.0.5,prompt,success,210,1020
2025-01-01T10:00:10Z,charlie,10.0.0.7,login,failure,80,400
2025-01-01T10:00:15Z,bob,10.0.0.6,prompt,success,150,900
2025-01-01T10:00:20Z,charlie,10.0.0.7,prompt,failure,300,1100
2025-01-01T10:00:25Z,alice,10.0.0.5,login,success,105,700
2025-01-01T10:00:30Z,bob,10.0.0.6,login,success,115,650
2025-01-01T10:00:35Z,charlie,10.0.0.7,login,failure,90,500
2025-01-01T10:00:40Z,alice,10.0.0.5,prompt,success,180,980
2025-01-01T10:00:45Z,bob,10.0.0.6,prompt,failure,260,1300
2025-01-01T10:00:50Z,charlie,10.0.0.7,prompt,success,190,850
''')

csv_path = 'sample_events.csv'
with open(csv_path, 'w', encoding='utf-8') as f:
    f.write(csv_data.getvalue())

print('Wrote synthetic CSV to', csv_path)

## Load the CSV
Key steps when loading logs:
1) Set the path.
2) Parse timestamps.
3) Inspect for schema, missing values, and dtypes.

In [ ]:
# Load CSV into DataFrame
df = pd.read_csv(csv_path, parse_dates=['timestamp'])

print('DataFrame loaded. Rows:', len(df))
df.head()

## Inspect Structure
`head()`: quick peek at rows.
`describe()`: numeric summary (latency, tokens).
`info()`: columns, non-null counts, dtypes.

In [ ]:
print('--- head() ---')
display(df.head(5))

print('
--- describe() ---')
display(df.describe())

print('
--- info() ---')
df.info()

## Filtering Rows
Common tasks in SIEM/log triage:
- Focus on one user or IP.
- Look at failures only.
- Time-slice by window.

In [ ]:
# Example filters
failures = df[df['status'] == 'failure']
alice_events = df[df['user'] == 'alice']
recent = df[df['timestamp'] >= '2025-01-01T10:00:20Z']

print('Failures count:', len(failures))
print('Alice events count:', len(alice_events))
print('Recent events count:', len(recent))

display(failures.head())

## Group By and Aggregate
Group to summarize by user, action, or status. Examples:
- Count attempts per user.
- Average latency per action.
- Failure rate by user.

In [ ]:
# Attempts per user
attempts_per_user = df.groupby('user').size().rename('count').reset_index()
display(attempts_per_user)

# Average latency by action
latency_by_action = df.groupby('action')['latency_ms'].mean().reset_index()
display(latency_by_action)

# Failure rate by user
fail_rate = (
    df.assign(failed=df['status'] == 'failure')
      .groupby('user')['failed']
      .mean()
      .reset_index()
      .rename(columns={'failed': 'failure_rate'})
)
display(fail_rate)

## Why This Mirrors Real SIEM / Access Log Analysis
- **head/describe/info**: sanity-check schema and quality.
- **Filtering**: isolate one principal, IP, or timeframe.
- **Grouping**: surface hot accounts, slow actions, or high failure rates.
- **Latency/tokens**: in prompt logs, track performance and usage.

These steps generalize to SOC workflows: triage anomalies, pivot on entities, summarize trends.

## Student TODO: Load Your Own CSV
Replace the path with your file (e.g., exported SIEM slice). Parse timestamps and inspect.

In [ ]:
# TODO: Point to your CSV file
your_csv_path = 'REPLACE_ME.csv'  # e.g., '/path/to/logs.csv'
# df_custom = pd.read_csv(your_csv_path, parse_dates=['timestamp'])
# display(df_custom.head())
# df_custom.info()

## Student TODO: Filters & Grouping
Practice with your dataset or the synthetic one above.

In [ ]:
# TODO: Write your own filters
# Example ideas:
# 1) Filter by a specific user or IP
# 2) Filter by status == 'failure'
# 3) Filter by a time window

# user_filter = df[df['user'] == 'alice']
# failures_only = df[df['status'] == 'failure']
# time_window = df[df['timestamp'] >= '2025-01-01T10:00:20Z']

# TODO: Write your own group-by
# Example ideas:
# - Count per user/action/status
# - Avg latency per action or user
# - Failure rate per user

# attempts = df.groupby('user').size()
# latency = df.groupby('action')['latency_ms'].mean()
# fail_rate = (df.assign(failed=df['status']=='failure')
#                 .groupby('user')['failed']
#                 .mean())

# Replace the examples above with your code and display the results.

## Student TODO: Interpretation
Answer in text (no code required):
1) Which users or actions show the highest failure rates?
2) Are there spikes in latency that might indicate service issues?
3) How would you pivot if you saw a suspicious IP or user?
4) What additional fields would you want (e.g., geo, device, user agent)?

---
**Takeaways**
- Load → Inspect → Filter → Aggregate is the core investigative loop.
- The same patterns apply to auth logs, prompt telemetry, or SIEM exports.
- Small, fast summaries help you spot anomalies before deep dives.